In [1]:
import pandas as pd
import os
# %pip install openpyxl
import openpyxl as openpyxl

In [2]:
# Folder with all your Excel files
folder_path = "../BoS (Business Objects) Raw Data Reports - Deidentified"


start_date = pd.to_datetime('10/11/2023', format='%m/%d/%Y')
end_date = pd.to_datetime('10/10/2024', format='%m/%d/%Y')

In [3]:
def percent_income_yes(folder_path, start_date, end_date):
    """
    This function filters the income at entry data based on the date range provided. It then creates an excel file with the percent of 
    participants who report having income at entry and exit. 
    """
    results = [] # Store all the rows
    
    # Loop through all Excel files in the folder
    for file in os.listdir(folder_path):
        if file == "TEMPLATE RAW Client Data Export v3_EE Workflow.xlsx":
            continue  # Skip this specific file
        
        if file.endswith(".xlsx"):
            file_path = os.path.join(folder_path, file)
            program_name = file.split(" ")[0]  # Extract 'Program' from filename

        try:
            #read in the excel files, accessing the sheets we need
            income_ent = pd.read_excel(file_path, sheet_name='INCOME ENT')
            income_ext = pd.read_excel(file_path, sheet_name='INCOME EXT')
        except Exception as e:
            print(f"Error reading sheets from {file}: {e}")
            continue

        #converting the date columns to datetime format and filtering the data based on the date range
        income_ent['Income Start Date (Entry)'] = pd.to_datetime(income_ent['Income Start Date (Entry)'], format='%m/%d/%Y', errors='coerce')
        filtered_income_ent = income_ent[(income_ent['Income Start Date (Entry)'] >= start_date) & (income_ent['Income Start Date (Entry)'] <= end_date)]

        income_ext['Income Start Date (Exit)'] = pd.to_datetime(income_ext['Income Start Date (Exit)'], format='%m/%d/%Y', errors='coerce')
        filtered_income_ext = income_ext[(income_ext['Income Start Date (Exit)'] >= start_date) & (income_ext['Income Start Date (Exit)'] <= end_date)]

        # Sum the number of 'Yes' values in the 'Receiving Income (Entry)' column and divide by the total number of rows in the filtered dataframe
        # Calculate % Yes for entry
        total_ent = len(filtered_income_ent)
        yes_ent = filtered_income_ent['Receiving Income (Entry)'].value_counts().get('Yes', 0)
        percent_ent = (yes_ent / total_ent * 100) if total_ent > 0 else 0

        # Calculate % Yes for exit
        total_ext = len(filtered_income_ext)
        yes_ext = filtered_income_ext['Receiving Income (Exit)'].value_counts().get('Yes', 0)
        percent_ext = (yes_ext / total_ext * 100) if total_ext > 0 else 0

        results.append({
            'Program': program_name,
            'Percent income at entry YES': round(percent_ent, 2),
            'Percent income at exit YES': round(percent_ext, 2)
        })
    
        # Save results to excel
        results_df = pd.DataFrame(results);
        results_df.to_excel("./Percent Income Entry & Exit.xlsx", index=False)
        # print(results_df)

In [4]:
percent_income_yes(folder_path, start_date, end_date)

In [11]:
def income_counts_entry_and_exit(folder_path, start_date, end_date):
    """
    This function filters the income at entry and exit data based on the date range provided. It creates
    a dataframe with the counts of 'Yes' and 'No' values for each program at entry and exit.
    The function also prints the value counts for each program and saves the results to an excel file.
    """
    results = [] # Store all the rows

    #Loop through all Excel files in the folder
    for file in os.listdir(folder_path):
        if file == "TEMPLATE RAW Client Data Export v3_EE Workflow.xlsx":
            continue

        if file.endswith(".xlsx"):
            file_path = os.path.join(folder_path, file)
            program_name = file.split(" ")[0]  # Extract 'Program' from filename

        try:
            income_ent = pd.read_excel(file_path, sheet_name='INCOME ENT')
            income_ext = pd.read_excel(file_path, sheet_name='INCOME EXT')
        except Exception as e:
            print(f"Error reading sheets from {file}: {e}")
            continue

        #converting the date columns to datetime format and filtering the data based on the date range
        income_ent['Income Start Date (Entry)'] = pd.to_datetime(income_ent['Income Start Date (Entry)'], format='%m/%d/%Y', errors='coerce')
        filtered_income_ent = income_ent[(income_ent['Income Start Date (Entry)'] >= start_date) & (income_ent['Income Start Date (Entry)'] <= end_date)]

        income_ext['Income Start Date (Exit)'] = pd.to_datetime(income_ext['Income Start Date (Exit)'], format='%m/%d/%Y', errors='coerce')
        filtered_income_ext = income_ext[(income_ext['Income Start Date (Exit)'] >= start_date) & (income_ext['Income Start Date (Exit)'] <= end_date)]

        yes_count_entry = filtered_income_ent['Receiving Income (Entry)'].value_counts().get('Yes', 0)
        no_count_entry = filtered_income_ent['Receiving Income (Entry)'].value_counts().get('No', 0)
        empty_count_entry = filtered_income_ent['Receiving Income (Entry)'].value_counts().get('', 0)

        yes_count_exit = filtered_income_ext['Receiving Income (Exit)'].value_counts().get('Yes', 0)
        no_count_exit = filtered_income_ext['Receiving Income (Exit)'].value_counts().get('No', 0)
        empty_count_exit = filtered_income_ext['Receiving Income (Exit)'].value_counts().get('', 0)
        #add new rows to the income_count_results dataframe
        new_entry_row_yes = ({'Receiving Income': 'Yes', 'Count': yes_count_entry, 'Assessment Stage': 'Entry', 'Program': program_name})
        new_entry_row_no = ({'Receiving Income': 'No', 'Count': no_count_entry, 'Assessment Stage': 'Entry', 'Program': program_name})
        new_entry_row_empty = ({'Receiving Income': 'Empty', 'Count': empty_count_entry, 'Assessment Stage': 'Entry', 'Program': program_name})

        new_exit_row_yes = ({'Receiving Income': 'Yes', 'Count': yes_count_exit, 'Assessment Stage': 'Exit', 'Program': program_name})
        new_exit_row_no = ({'Receiving Income': 'No', 'Count': no_count_exit, 'Assessment Stage': 'Exit', 'Program': program_name})
        new_exit_row_empty = ({'Receiving Income': 'Empty', 'Count': empty_count_exit, 'Assessment Stage': 'Exit', 'Program': program_name})

        #append the new rows to the results list
        results.append(new_entry_row_yes)
        results.append(new_entry_row_no)
        results.append(new_entry_row_empty)
        results.append(new_exit_row_yes)
        results.append(new_exit_row_no)
        results.append(new_exit_row_empty)

        # Save results to excel
        results_df = pd.DataFrame(results);
        results_df.to_excel("./Income Counts Entry and Exit.xlsx", index=False)


In [12]:
income_counts_entry_and_exit(folder_path, start_date, end_date)

In [17]:
def percent_noncash_yes(folder_path, start_date, end_date):
    """
    This function filters the non-cash data based on the date range provided. It then creates an excel file with the percent of 
    participants who report receiving non-cash benefits at entry and exit.
    """
    results = [] # Store all the rows

    #Loop through all Excel files in the folder
    for file in os.listdir(folder_path):
        if file == "TEMPLATE RAW Client Data Export v3_EE Workflow.xlsx":
            continue

        if file.endswith(".xlsx"):
            file_path = os.path.join(folder_path, file)
            program_name = file.split(" ")[0]  # Extract 'Program' from filename

        try:
            noncash_ent = pd.read_excel(file_path, sheet_name='NONCASH ENT')
            noncash_ext = pd.read_excel(file_path, sheet_name='NONCASH EXIT')

        except Exception as e:
            print(f"Error reading sheets from {file}: {e}")
            continue

        #converting the date columns to datetime format and filtering the data based on the date range
        noncash_ent['Non-Cash Start Date (Entry)'] = pd.to_datetime(noncash_ent['Non-Cash Start Date (Entry)'], format='%m/%d/%Y', errors='coerce')
        filtered_noncash_ent = noncash_ent[(noncash_ent['Non-Cash Start Date (Entry)'] >= start_date) & (noncash_ent['Non-Cash Start Date (Entry)'] <= end_date)]

        noncash_ext['Non-Cash Start Date (Exit)'] = pd.to_datetime(noncash_ext['Non-Cash Start Date (Exit)'], format='%m/%d/%Y', errors='coerce')
        filtered_noncash_ext = noncash_ext[(noncash_ext['Non-Cash Start Date (Exit)'] >= start_date) & (noncash_ext['Non-Cash Start Date (Exit)'] <= end_date)]

        #sum the number of 'Yes' values in the 'Receiving Income (Entry)' column and divide by the total number of rows in the filtered dataframe
        total_rows_entry = len(filtered_noncash_ent)
        if total_rows_entry > 0:
            num_yes = filtered_noncash_ent['Receiving Benefit (Entry)'].value_counts().get('Yes', 0)
            percent_income_at_entry = (num_yes / total_rows_entry * 100)
        else:
            percent_income_at_entry = 0

        #repeat the same process for the 'Receiving Income (Exit)' column
        total_rows_exit = len(filtered_noncash_ext)
        if total_rows_exit > 0:
            num_yes_exit = filtered_noncash_ext['Receiving Benefit (Exit)'].value_counts().get('Yes', 0)
            percent_noncash_at_exit = (num_yes_exit / total_rows_exit * 100)
        else:
            percent_noncash_at_exit = 0

        #add the program and percent_income_at_entry to the income_ent_results dataframe
        new_entry_row = ({'Program': program_name, 'Percent non-cash at entry YES': round(percent_income_at_entry, 2), 'Percent non-cash at exit YES': round(percent_noncash_at_exit, 2)})
        results.append(new_entry_row)

        # Save results to excel
        results_df = pd.DataFrame(results);
        results_df.to_excel("./Percent Non-Cash Yes at Entry and Exit.xlsx", index=False)

In [18]:
percent_noncash_yes(folder_path, start_date, end_date)

In [ ]:
def calculate_median_income_everyone(list_of_programs, file_names, start_date, end_date, file_path_for_output_files):
    """This function calculates the median income at entry and exit for each program for everybody (those who report having
    income at entry and exit, and those who don't). 
    It filters the data based on the date range provided and creates an excel file with the results."""
    median_income_results = pd.DataFrame(columns=['Program', 'Median Income at Entry', 'Median Income at Exit'])
    #read in the excel files, accessing the sheets we need
    for program, file_name in zip(list_of_programs, file_names):
        income_ent = pd.read_excel(file_name, sheet_name='INCOME ENT')
        income_ext = pd.read_excel(file_name, sheet_name='INCOME EXT')

        #converting the date columns to datetime format and filtering the data based on the date range
        income_ent['Income Start Date (Entry)'] = pd.to_datetime(income_ent['Income Start Date (Entry)'], format='%m/%d/%Y', errors='coerce')
        filtered_income_ent = income_ent[(income_ent['Income Start Date (Entry)'] <= end_date) & (income_ent['Income Start Date (Entry)'] >= start_date)]

        income_ext['Income Start Date (Exit)'] = pd.to_datetime(income_ext['Income Start Date (Exit)'], format='%m/%d/%Y', errors='coerce')
        filtered_income_ext = income_ext[(income_ext['Income Start Date (Exit)'] <= end_date) & (income_ext['Income Start Date (Exit)'] >= start_date)]

        #calculate median income at entry and exit
        median_income_entry = filtered_income_ent['Monthly Income (Start)'].median()
        median_income_exit = filtered_income_ext['Monthly Income (Start)'].median()

        #add the program and median incomes to the results dataframe
        new_entry_row = pd.DataFrame({'Program': [program], 'Median Income at Entry': median_income_entry, 'Median Income at Exit': median_income_exit})
        median_income_results = pd.concat([median_income_results, new_entry_row], ignore_index=True)

In [ ]:
calculate_median_income_everyone(list_of_programs, file_names, start_date, end_date, file_path_for_output_files)

FileNotFoundError: [Errno 2] No such file or directory: '143 RAW Client Data Export v3_EE Workflow.xlsx'

To Do:
- (COMPLETED, THOUGH LIKELY NOT ACCURATE. Create a function that calculates the median monthly income for each program a entry and exit
- (COMPLETTED) Modify the percent_income_yes() function to create a new function that calculates the percent of program participants receiving non cash benefits. We should only need to update the sheet name and confirm the column name is the same in that sheet. 

If there is time:
- Create a function that calculates the average monthly program at entry and exit
- Create a function that sums the type of employment/income sources by program. This should be similar to the income_counts_entry_and_exit() function.
- They requested a metric that showed new or increased income. This one is an optional request because they don't know if it can be calculated. 
